Let's specify the different specificities of our modell. We have to define the dimensions, and the specificities of each material:

In [2]:
!pip install numpy pandas matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 69.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 114.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 107.3 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

l = 3               # m length of the room
L=5                 #Largeur of the room
H=2.5               #Height of the room
Hw=1.5              #Hauteur Window
Lw=0.9              #Longueur Window
Sp=3                #Surface porte
Sw = 2*Hw*Lw        # m² surface area of the windows
Sc = L*H+2*l*H-Sp+L*l   # m² surface area of plasterboard
Sout = L*H-Sw       # m² surface area of concrete & insulation of the walls
To=5
Ti=20
Tiop=22
Kp=100

air = {'Density': 1.2,                      # kg/m³
       'Specific heat': 1000}               # J/(kg·K)
pd.DataFrame(air, index=['Air'])

concrete = {'Conductivity': 1.400,          # W/(m·K)
            'Density': 2300.0,              # kg/m³
            'Specific heat': 880,           # J/(kg⋅K)
            'Width': 0.2,                   # m
            'Surface': L*H-Sw}            # m²

insulation = {'Conductivity': 0.027,        # W/(m·K)
              'Density': 55.0,              # kg/m³
              'Specific heat': 1210,        # J/(kg⋅K)
              'Width': 0.08,                # m
              'Surface': L*H-Sw}          # m²

plasterboard = {'Conductivity': 0.25,        # W/(m·K)
              'Density': 9.3,              # kg/m³
              'Specific heat': 500,        # J/(kg⋅K)
              'Width': 0.05,                # m
              'Surface': L*H+2*l*H-Sp+L*l}          # m²

glass = {'Conductivity': 1.4,               # W/(m·K)
         'Density': 2500,                   # kg/m³
         'Specific heat': 1210,             # J/(kg⋅K)
         'Width': 0.04,                     # m
         'Surface': Hw*Lw}                   # m²

wall = pd.DataFrame.from_dict({'Layer_out': concrete,
                               'Layer_in': insulation, 'Plasterboard':plasterboard,
                               'Glass': glass},
                              orient='index')

# radiative properties
ε_wLW = 0.85    # long wave emmisivity: wall surface (concrete)
ε_gLW = 0.90    # long wave emmisivity: glass pyrex
ε_pLW = 0.90    # long wave emmisivity: plasterboard
α_wSW = 0.25    # short wave absortivity: white smooth surface
α_gSW = 0.38    # short wave absortivity: reflective blue glass
τ_gSW = 0.30    # short wave transmitance: reflective blue glass

σ = 5.67e-8     # W/(m²⋅K⁴) Stefan-Bolzmann constant

h = pd.DataFrame([{'in': 8., 'out': 25}], index=['h'])  # W/(m²⋅K)

Thanks to the thermal circuit, we can define the matrix A as following:


In [4]:
A= np.array([
    [ 1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
    [-1,  1,  0,  0,  0,  0,  0,  0,  0,  0],
    [ 0, -1,  1,  0,  0,  0,  0,  0,  0,  0],
    [ 0,  0, -1,  1,  0,  0,  0,  0,  0,  0],
    [ 0,  0,  0, -1,  1,  0,  0,  0,  0,  0],
    [ 0,  0,  0,  0, -1,  0,  1,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  1, -1,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  0,  1,  0,  0],
    [ 0,  0,  0,  0,  0,  1, -1,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  0,  1,  0,  0, -1],
    [ 0,  0,  0,  0,  0,  1,  0,  0, 0,  -1],
    [ 0,  0,  0,  0, -1,  1,  0,  0, 0,   0],
    [ 0,  0,  0,  0,  0,  0,  0,  0,  1, -1],
    [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
    [ 0,  0,  0,  0,  0,  1,  0,  0,  0,  0],
    [ 0,  0,  0,  0,  0,  1,  0,  0,  0,  0]
])

Then we can define the G matrix that corresponds to the conductances:

In [5]:
# conduction
G_cd = wall['Conductivity'] / wall['Width'] * wall['Surface']
pd.DataFrame(G_cd, columns=['Conductance'])

# convection
Gw_out = h * wall['Surface'].iloc[0]     # wall
Gw_in = h * wall['Surface'].iloc[2]     # wall
Gg = h * wall['Surface'].iloc[3]     # glass

# view factor glass-wall
Fgw = 1

#Factor wall_out-wall_in

Fwo_wi=1

Tm = 20 + 273   # K, mean temp for radiative exchange

#Long Wave radiation between glass and plasterboard wall
GLW1_2 = 4 * σ * Tm**3 * ε_pLW / (1 - ε_pLW) * wall['Surface']['Plasterboard']
GLW12_2 = 4 * σ * Tm**3 * Fgw * wall['Surface']['Glass']
GLW2_2 = 4 * σ * Tm**3 * ε_gLW / (1 - ε_gLW) * wall['Surface']['Glass']
GLW_2 = 1 / (1 / GLW1_2 + 1 / GLW12_2 + 1 / GLW2_2)

#Long wave radiation between insulated wall and plasterboard wall
GLW1_1 = 4 * σ * Tm**3 * ε_pLW / (1 - ε_pLW) * wall['Surface']['Plasterboard']
GLW12_1 = 4 * σ * Tm**3 * Fwo_wi * wall['Surface']['Layer_in']
GLW2_1 = 4 * σ * Tm**3 * ε_pLW / (1 - ε_pLW) * wall['Surface']['Layer_in']
GLW_1 = 1 / (1 / GLW1_1 + 1 / GLW12_1 + 1 / GLW2_1)

G0=Gw_out['out'].iloc[0]            #Convection between outside and concrete wall
G1=G2=G_cd['Layer_out']/2    #conductions in the concrete wall
G3=G4=G_cd['Layer_in']/2  #conduction in the insulator
G5=GLW_1                    #Long Wave radiation between the wall (insulation) and the plasterboard wall
G9=GLW_2                    #Long Wave radiation between the plasterboard wall and the glass
G6=G7=G_cd['Plasterboard']/2 #Conduction in the plasterboard
G8= Gw_in['in'].iloc[0]             #Convection from plasterboard
G10=Gg['in'].iloc[0]               #Convection from glass
G11=Gw_out['in'].iloc[0]           #Convection from the insulator
G12=G_cd['Glass']/2         #Conduction in glass
Gglass_conv=Gg['out'].iloc[0]       #Convection on the outside Window
G13=G12*Gglass_conv/(G12+Gglass_conv) #conduction and convection in glass
Va = l*L*H                  # m³, volume of air
ACH = 1                     # 1/h, air changes per hour
Va_dot = ACH / 3600 * Va    # m³/s, air infiltration

G14= air['Density'] * air['Specific heat'] * Va_dot # Advection

Glist=[G0,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,Kp]
Garray=np.array(Glist)
G=np.diag(Garray)

We also define the C matrix that represents the thermal capacities, the f matrix, whix corresponds to the fluxes and b that defines the initial temperatures

In [6]:
# Temperature source vector
b = np.array([To, 0, 0, 0, 0, 0, 0, Ti, 0, 0, 0, 0, 0, To, Ti, Tiop])
f=np.zeros(10)
Phi_glass=300
Phi_concrete=1000
Phi_humans=200
f[[0]]=Phi_concrete
f[[5]]=Phi_humans
f[[9]]=Phi_glass

We now have all the information to determine the steady state temperatures.

In [7]:
θss = np.linalg.inv(A.T @ G @ A) @ (A.T @ G @ b + f)
print(θss)

[ 9.12467004  9.43207992  9.73948981 16.11539857 22.49130732 22.69431634
 22.38190046 21.19095023 24.33782686 24.33782686]
